In [1]:
! pip install langchain_groq
! pip install pymupdf


/bin/bash: /home/kb/MES/MES_POC/POC/venv_kri_mes/bin/pip: /home/kb/MES_Copilot/POC/venv_kri_mes/bin/python: bad interpreter: No such file or directory
/bin/bash: /home/kb/MES/MES_POC/POC/venv_kri_mes/bin/pip: /home/kb/MES_Copilot/POC/venv_kri_mes/bin/python: bad interpreter: No such file or directory


In [2]:
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain_community.document_loaders import PyMuPDFLoader

In [3]:
# Load documents
pdf_loader = PyMuPDFLoader("krishna_resume.pdf")
documents = pdf_loader.load()

# Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

vector_store = FAISS.from_documents(texts, embeddings)
retriever = vector_store.as_retriever()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)


/home/kb/MES/MES_POC/POC/venv_kri_mes/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# llm = ChatGroq(
#     model="mixtral-8x7b-32768",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2
# )


llm = ChatGroq(model="llama3-70b-8192", temperature=0)

In [5]:
@tool
def retrieve_documents(query: str) -> str:
    """Retrieve relevant documents based on the query"""
    docs = retriever.invoke(query)
    return "\n".join([doc.page_content for doc in docs])


In [6]:
tools = [retrieve_documents]
query = "who is Krishna?"

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": f"Retrieve relevant documents for: {query}"})




> Entering new AgentExecutor chain...

Invoking: `retrieve_documents` with `{'query': 'who is Krishna?'}`


KRISHNA DIPAYAN BHUNIA 
Senior Software Engineer | Senior Data Engineer | Python Developer 
+91 865 200 7894 
 
Github Link 
LinkedIn 
 
kri.career@gmail.com 
SUMMARY 
● Seasoned Software Engineer/Full Stack Developer with 12+ years of experience across industries like CRM, Oil & Gas, Financial, and FMCG. Over 
6 years of expertise in Python as a Data Engineer/Scientist, specializing in data engineering, analysis, and visualization.  
● Proficient in writing clean, production-level Python code, with strong skills in Object-Oriented Programming, DSA, and relational databases. Used 
Python libraries such as NumPy, and Pandas for advanced data analysis and engineering stuff. 
● Experienced in designing software solutions using Python, C#, and back-end technologies. Adept in cloud platforms (AWS, Azure), containerization 
(Docker), and database management (SQL, No-SQL), with a stro

{'input': 'Retrieve relevant documents for: who is Krishna?',
 'output': 'Krishna Dipayan Bhunia is a seasoned software engineer and data engineer with over 12 years of experience across various industries. He has expertise in Python, data engineering, analysis, and visualization. He has worked on several projects, including data pipelines, ETL, and data visualization, and has experience with cloud platforms, containerization, and database management.'}

In [7]:
tools = [retrieve_documents]
query = "In which company Krishna works?"

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": f"Retrieve relevant documents for: {query}"})



> Entering new AgentExecutor chain...

Invoking: `retrieve_documents` with `{'query': 'In which company Krishna works?'}`


KRISHNA DIPAYAN BHUNIA 
Senior Software Engineer | Senior Data Engineer | Python Developer 
+91 865 200 7894 
 
Github Link 
LinkedIn 
 
kri.career@gmail.com 
SUMMARY 
● Seasoned Software Engineer/Full Stack Developer with 12+ years of experience across industries like CRM, Oil & Gas, Financial, and FMCG. Over 
6 years of expertise in Python as a Data Engineer/Scientist, specializing in data engineering, analysis, and visualization.  
● Proficient in writing clean, production-level Python code, with strong skills in Object-Oriented Programming, DSA, and relational databases. Used 
Python libraries such as NumPy, and Pandas for advanced data analysis and engineering stuff. 
● Experienced in designing software solutions using Python, C#, and back-end technologies. Adept in cloud platforms (AWS, Azure), containerization 
(Docker), and database management (SQL, No-S

{'input': 'Retrieve relevant documents for: In which company Krishna works?',
 'output': 'Tool use failed: no valid JSON object found in message content'}

In [8]:
tools = [retrieve_documents]
query = "Pls answer in one word, where Krishna works?"

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": f"Retrieve relevant documents for: {query}"})




> Entering new AgentExecutor chain...

Invoking: `retrieve_documents` with `{'query': 'where Krishna works'}`


KRISHNA DIPAYAN BHUNIA 
Senior Software Engineer | Senior Data Engineer | Python Developer 
+91 865 200 7894 
 
Github Link 
LinkedIn 
 
kri.career@gmail.com 
SUMMARY 
● Seasoned Software Engineer/Full Stack Developer with 12+ years of experience across industries like CRM, Oil & Gas, Financial, and FMCG. Over 
6 years of expertise in Python as a Data Engineer/Scientist, specializing in data engineering, analysis, and visualization.  
● Proficient in writing clean, production-level Python code, with strong skills in Object-Oriented Programming, DSA, and relational databases. Used 
Python libraries such as NumPy, and Pandas for advanced data analysis and engineering stuff. 
● Experienced in designing software solutions using Python, C#, and back-end technologies. Adept in cloud platforms (AWS, Azure), containerization 
(Docker), and database management (SQL, No-SQL), with a 

{'input': 'Retrieve relevant documents for: Pls answer in one word, where Krishna works?',
 'output': 'Dwarka'}

In [9]:
tools = [retrieve_documents]
query = "where Krishna was working on 2012"

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": f"Retrieve relevant documents for: {query}"})




> Entering new AgentExecutor chain...

Invoking: `retrieve_documents` with `{'query': 'where Krishna was working on 2012'}`


KRISHNA DIPAYAN BHUNIA 
Senior Software Engineer | Senior Data Engineer | Python Developer 
+91 865 200 7894 
 
Github Link 
LinkedIn 
 
kri.career@gmail.com 
SUMMARY 
● Seasoned Software Engineer/Full Stack Developer with 12+ years of experience across industries like CRM, Oil & Gas, Financial, and FMCG. Over 
6 years of expertise in Python as a Data Engineer/Scientist, specializing in data engineering, analysis, and visualization.  
● Proficient in writing clean, production-level Python code, with strong skills in Object-Oriented Programming, DSA, and relational databases. Used 
Python libraries such as NumPy, and Pandas for advanced data analysis and engineering stuff. 
● Experienced in designing software solutions using Python, C#, and back-end technologies. Adept in cloud platforms (AWS, Azure), containerization 
(Docker), and database management (SQL, No

{'input': 'Retrieve relevant documents for: where Krishna was working on 2012',
 'output': 'Krishna Dipayan Bhunia was working as a Graduate Engineer Trainee in Godrej & Boyce in the CRM domain from October 2012 to September 2013.'}